이전 단계에서 생성한 incidents_val_minedHN.jsonl 파일을  
모델 테스트 데이터 포맷으로 변환 및 저장.  

incidents_ft_minedHN.jsonl은 그대로 사용가능하므로 냅둔다.

In [ ]:
from datasets import load_dataset, Features, Value

fpath = "../../dataset/fine-tuning/incidents_val_minedHN.jsonl"
ds = load_dataset('json', data_files=fpath)['train']

# 새 피처 정의
new_features = Features({
    'id': Value('string'),
    'title': ds.features['title'],
    'query': ds.features['query'],
    'pos': ds.features['pos'],
    'neg': ds.features['neg'],
})


# def int_to_str(row):
#     row['id'] = str(row['id'])
#     return row

# ds = ds.map(int_to_str, features=new_features)

ds = ds.map(features=new_features)
ds[0]

c:\Users\fehs0\miniconda3\envs\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 363 examples [00:00, 20277.45 examples/s]
Map: 100%|██████████| 363/363 [00:00<00:00, 10649.24 examples/s]


{'id': '219',
 'title': '형법 제87조',
 'query': '제87조(내란) 대한민국 영토의 전부 또는 일부에서 국가권력을 배제하거나 국헌을 문란하게 할 목적으로 폭동을 일으킨 자는 다음 각 호의 구분에 따라 처벌한다',
 'pos': ['피고인들이 집단을 동원해 국가기관의 정상적 운영을 중단시키기 위해 행정기관 출입을 봉쇄한 사실',
  '무력으로 세관과 국경을 점거하여 국가의 영토 통제를 사실상 유린하려 한 사실',
  '피고인들이 국가 지도자의 암살을 시도하여 국헌을 문란하게 할 목적으로 폭력을 행사한 사실',
  '집단이 정부에 대항하는 무장세력을 조직하고 외국세력의 군사적 지원을 받아 공격을 감행한 사실',
  '피고인들이 주요 공공기록과 행정문서를 파괴해 정부 업무 수행을 불가능하게 하려 한 사실',
  '무장한 집단이 주둔군과 충돌을 일으켜 군의 지휘체계를 붕괴시키려 한 사실',
  '피고인들이 무력을 동원해 입법기관의 개회를 강제로 저지하고 입법권 행사를 방해한 사실',
  '집단이 경찰과 군의 이동을 저지하기 위해 도로와 교량을 폭파하려 한 사실',
  '피고인들이 지역 내에서 자체 행정·사법 기구를 설치하고 주민들에게 복종을 강요한 사실',
  '조직적으로 공공시설의 전력·통신망을 파괴하여 국가 기능 전반을 마비시키려 한 사실'],
 'neg': ['피고인은 국기의 도안 위에 특정 인물에 대한 모욕적 문구를 합성해 유포함으로써 국기를 비방하였다.',
  '피고인은 국가적 비상사태 기간 중 중요문화재 보관창고에 폭발물을 투입하여 귀중한 재산을 소실시키고 사회적 혼란을 초래한 혐의로 기소되었다.',
  '피고인은 피해자를 강제로 차량에서 내리지 못하게 하고 다수의 폭행과 폭언으로 이동을 제한하였다.',
  '기소장에 따르면 피고인은 청탁의 대가로 현물(귀금속·명품시계)을 제3자 H에게 보관하도록 하였고 그 물건들은 제134조에 따라 몰수되어야 한다.',
  '피고인은 적국을 위하여 군용 차량의 브레이크 라인을 절단해 운용을 

In [2]:
queries = ds.select_columns(['id', 'query']).rename_column('query', 'text')
corpus = ds.select_columns(['id', 'pos']).rename_column('pos', 'text')

qrels = ds.select_columns(['id']).rename_column('id', 'qid')

qrels = qrels.add_column('docid', ds['id'])
qrels = qrels.add_column('relevance', [1] * len(ds)) # 연관도 수치는 1로 고정

In [3]:
queries[1]

{'id': '220', 'text': '우두머리는 사형 무기징역 또는 무기금고에 처한다'}

In [4]:
corpus[1]

{'id': '220',
 'text': ['피고인은 국가의 치안을 무너뜨릴 목적으로 유사군사조직을 구성·지휘하였다.',
  '피고인은 집단행동을 조직하여 탄약과 폭발물을 사용해 다수의 공공시설을 파괴하였다.',
  '피고인은 반정부 선언을 발표하고 조직원의 자발적 공격을 조장하여 폭력사태를 촉발했다.',
  '피고인은 폭동과 관련한 피해자들의 생명과 재산에 중대한 피해를 발생시키는 구체적·계획적 행위를 지휘했다.',
  '피고인은 조직원에게 살상무기 사용을 허용하며 적극적으로 공격행위를 독려하였다.',
  '피고인은 시점과 목표를 선정하여 조직적인 충돌을 계획·지휘한 정황이 수사과정에서 확인되었다.',
  '피고인은 폭력적 봉기의 최고책임자로 지목되며 다수의 공범을 동원하여 정부기구를 공격하였다.',
  '피고인은 공격 명령을 직접 하달하고, 평화적 집회가 무력충돌로 번지도록 조율하였다.',
  '피고인은 피해자 진술과 CCTV에 의해 조직적·지휘적 범행 사실이 확인되어 지도자로 기소되었다.',
  '피고인은 조직의 우두머리로서 범죄행위에 대한 책임을 인정받아 중형이 구형된 상태이다.']}

In [5]:
qrels[1]

{'qid': '220', 'docid': '220', 'relevance': 1}

In [6]:
queries.to_json("../../dataset/test/gpt_test_queries.jsonl", force_ascii=False)
corpus.to_json("../../dataset/test/gpt_test_corpus.jsonl", force_ascii=False)
qrels.to_json("../../dataset/test/gpt_test_qrels.jsonl", force_ascii=False)

Creating json from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 897.18ba/s]


15246